In [6]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tiff
from typing import Tuple, Union

def _to_numpy_gray(img_or_path: Union[str, np.ndarray]) -> np.ndarray:
    """Load image if path; ensure 2D float array in [0,1] for display."""
    if isinstance(img_or_path, str):
        img = tiff.imread(img_or_path)
    else:
        img = np.asarray(img_or_path)
    # If it's a stack, take the middle slice; if RGB(A), convert to luminance
    if img.ndim == 3 and img.shape[-1] not in (3,4):
        img = img[img.shape[0]//2]
    if img.ndim == 3 and img.shape[-1] in (3,4):
        img = img[..., :3].mean(axis=-1)
    img = img.astype(np.float64, copy=False)
    lo, hi = np.percentile(img, [0.5, 99.5])
    if hi <= lo:
        lo, hi = img.min(), (img.max() if img.max() > img.min() else 1.0)
    return np.clip((img - lo) / (hi - lo + 1e-12), 0.0, 1.0)

def _crop_center_with_pad(img: np.ndarray, cx: float, cy: float, W: int) -> Tuple[np.ndarray, int, int]:
    """Crop a W×W region centered at (cx, cy). Returns (crop, x0, y0) in original coords."""
    pad = int(np.ceil(W/2) + 2)
    imgp = np.pad(img, pad_width=pad, mode='edge')
    cxp, cyp = cx + pad, cy + pad
    x0 = int(np.floor(cxp - W/2))
    y0 = int(np.floor(cyp - W/2))
    crop = imgp[y0:y0+W, x0:x0+W]
    return crop, x0 - pad, y0 - pad

def plot_10(ori, RL, Sp, ZS, ZS_R, x1, y1, x2, y2, W):
    """
    Plot five images + five zooms. Draw the line (x1,y1)-(x2,y2) on all subplots.
    Returns (fig, axes) with axes shape (2,5).
    """
    titles = ["Original", "RL", "Sparse", "ZS-Net", "ZS-Net (reg)"]
    imgs = [_to_numpy_gray(im) for im in [ori, RL, Sp, ZS, ZS_R]]

    # Midpoint for zoom
    cx = 0.5 * (x1 + x2)
    cy = 0.5 * (y1 + y2)
    W = int(W)

    fig, axes = plt.subplots(2, 5, figsize=(15, 6))

    # Top row: full images
    for ax, img, title in zip(axes[0], imgs, titles):
        ax.imshow(img, cmap='gray', origin='upper')
        ax.set_title(title)
        ax.plot([x1, x2], [y1, y2])  # line between points
        # show zoom box
        x0 = int(np.floor(cx - W/2)); y0 = int(np.floor(cy - W/2))
        rect = plt.Rectangle((x0, y0), W, W, fill=False, linestyle='--')
        ax.add_patch(rect)
        ax.set_axis_off()

    # Bottom row: zooms
    for ax, img, title in zip(axes[1], imgs, titles):
        crop, x0, y0 = _crop_center_with_pad(img, cx, cy, W)
        ax.imshow(crop, cmap='gray', origin='upper')
        ax.plot([x1 - x0, x2 - x0], [y1 - y0, y2 - y0])  # same line in crop coords
        ax.set_title(f"Zoom: {title}")
        ax.set_axis_off()

    plt.tight_layout()
    return fig, axes

def read_tif_norm(path, mode="percentile", p_low=0.5, p_high=99.5, slice_mode=None):
    """
    Read a .tif and normalize to [0,1].
      - mode: "percentile" (default) or "minmax"
      - slice_mode: None -> return as-is (2D or 3D stack)
                    "mid" -> return middle slice if a stack
                    "first" -> return first slice if a stack
    """
    img = tiff.imread(path)

    # If RGB(A), convert to grayscale
    if img.ndim == 3 and img.shape[-1] in (3, 4):
        img = img[..., :3].mean(axis=-1)

    # Optionally select a slice for stacks (Z,Y,X)
    if slice_mode in ("mid", "first") and img.ndim == 3 and img.shape[-1] not in (3,4):
        z = 0 if slice_mode == "first" else img.shape[0] // 2
        img = img[z]

    img = img.astype(np.float32, copy=False)

    if mode == "minmax":
        lo, hi = float(img.min()), float(img.max())
    else:  # "percentile"
        lo, hi = np.percentile(img, [p_low, p_high])
        if hi <= lo:  # fallback if image is flat
            lo, hi = float(img.min()), float(img.max()) if img.max() > img.min() else (0.0, 1.0)

    img = np.clip((img - lo) / (hi - lo + 1e-12), 0.0, 1.0)
    return img

AttributeError: module 'matplotlib' has no attribute '__version_info__'

In [4]:
ori = read_tif_norm(r"E:\Christopher\ZSD\Final\ori.tif", mode="percentile", p_low=0, p_high=100, slice_mode=None)
imshow

AttributeError: module 'matplotlib' has no attribute 'imshow'